In [1]:
!pip install langchain openai langchain-community langchain-openai beautifulsoup4

In [3]:
import os
from google.colab import userdata

# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# print(os.environ['OPENAI_API_KEY'])

KeyError: 'OPENAI_API_KEY'

In [11]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
import bs4
import json


# 네이버 뉴스기사 주소
# url = 'https://n.news.naver.com/mnews/article/025/0003311700?sid=101'
# url = 'https://velog.io/@isanghao/테스트는-단순-업무가-아니다-테크-리드가-직접-해야-하는-이유'
url = 'https://docs.nestjs.com/custom-decorators'

# 웹 문서 크롤링
loader = WebBaseLoader(
    web_path=url,
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("newsct_article _article_body")
    #     )
    # ),
)

# 뉴스기사의 본문을 Chunk 단위로 쪼갬
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

print(len(WebBaseLoader(url).load()))

# 웹사이트 내용 크롤링 후 Chunk 단위로 분할
docs = WebBaseLoader(url).load_and_split(text_splitter)
docs

1


[Document(metadata={'source': 'https://docs.nestjs.com/custom-decorators', 'title': 'Documentation | NestJS - A progressive Node.js framework', 'description': 'Nest is a framework for building efficient, scalable Node.js server-side applications. It uses progressive JavaScript, is built with TypeScript and combines elements of OOP (Object Oriented Programming), FP (Functional Programming), and FRP (Functional Reactive Programming).', 'language': 'en'}, page_content='Documentation | NestJS - A progressive Node.js framework')]

In [5]:
# 각 Chunk 단위의 템플릿
template = '''다음의 내용을 한글로 요약해줘:

{text}
'''

# 전체 문서(혹은 전체 Chunk)에 대한 지시(instruct) 정의
combine_template = '''{text}

요약의 결과는 다음의 형식으로 작성해줘:
제목: 신문기사의 제목
주요내용: 한 줄로 요약된 내용
작성자: 김철수 대리
내용: 주요내용을 불렛포인트 형식으로 작성
'''

# 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])
print(prompt)
print(combine_prompt)

input_variables=['text'] input_types={} partial_variables={} template='다음의 내용을 한글로 요약해줘:\n\n{text}\n'
input_variables=['text'] input_types={} partial_variables={} template='{text}\n\n요약의 결과는 다음의 형식으로 작성해줘:\n제목: 신문기사의 제목\n주요내용: 한 줄로 요약된 내용\n작성자: 김철수 대리\n내용: 주요내용을 불렛포인트 형식으로 작성\n'


In [6]:
# LLM 객체 생성
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-5-mini')

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# 요약을 도와주는 load_summarize_chain
chain = load_summarize_chain(llm,
                             map_prompt=prompt,
                             combine_prompt=combine_prompt,
                             chain_type='map_reduce',
                             verbose=False)

# 실행결과
output_text = chain.invoke(docs).get('output_text')
print(output_text)